# Otimização do Mix de Culturas — Versão com Hydra e Dash
Esta versão integra Hydra para gerenciamento de configurações YAML e Plotly/Dash para visualização interativa de cenários de otimização.

In [1]:
# 1. Install Required Libraries
# !pip install hydra-core plotly dash pandas numpy pulp

In [2]:
# 2. Setup Hydra Configuration
# Criar arquivos YAML manualmente ou via código
import os
from datetime import datetime
os.makedirs('config/cenario', exist_ok=True)

def print_and_log(message, logfile='outputs-otimizacao_mix_culturas_v3_gestao_risco.txt'):
    """Imprime no stdout e registra em um único arquivo de log (UTF-8).
    Consolidado em `outputs-otimizacao_mix_culturas_v3_gestao_risco.txt` conforme solicitado.
    """
    ts = datetime.now().isoformat(sep=' ', timespec='seconds')
    line = f'[{ts}] {message}'
    print(line)
    # log principal único (UTF-8)
    with open(logfile, 'a', encoding='utf-8') as f:
        f.write(line + '\n')

# base.yaml
base_config = """
resources:
  AREA_TOTAL_DISPONIVEL_HA: 500
  ORCAMENTO_TOTAL_DISPONIVEL: 1100000
  AGUA_TOTAL_DISPONIVEL_M3: 250000
  POTASSIO_DISPONIVEL_KG: 45000
  FOSFORO_DISPONIVEL_KG: 42000
  AREA_NAO_COMPACTADA_HA: 400
  HORAS_MAQUINA_DISPONIVEIS: 6000
  CAPACIDADE_SILO_TON: 2500

params:
  n_dados: 1500
  seed: 42
  preco_soja: 2200
  preco_milho: 1300
  percentual_minimo_por_cultura: 0.15
  percentual_maximo_soja_produtiva: 0.60
"""

with open('config/cenario/base.yaml', 'w', encoding='utf-8') as f:
    f.write(base_config)

# seca.yaml
seca_config = """
# @package _group_
resources:
  AGUA_TOTAL_DISPONIVEL_M3: 180000
"""

with open('config/cenario/seca.yaml', 'w', encoding='utf-8') as f:
    f.write(seca_config)

print_and_log('Config files created.')


[2025-10-16 15:42:43] Config files created.


In [3]:
# 3. Define Optimization Model with Hydra
import pandas as pd
import numpy as np
import pulp
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, value, LpStatus
from hydra import (
    compose,
    initialize,
)  # fix: use initialize instead of initialize_config_store
from omegaconf import DictConfig


def generate_mock_data(n_dados=1500, seed=42):
    # Mesmo código do original
    np.random.seed(seed)
    dados = {
        "id_talhao": range(1, n_dados + 1),
        "cultura": np.random.choice(
            ["Soja_Resistente", "Soja_Produtiva", "Milho_Safrinha"], n_dados
        ),
    }
    df = pd.DataFrame(dados)

    culturas_params = {
        "Soja_Resistente": {
            "prod": (3.5, 0.5),
            "custo": (1800, 150),
            "agua": (450, 50),
            "k": (80, 10),
            "p": (70, 8),
            "horas": (10, 1.5),
        },
        "Soja_Produtiva": {
            "prod": (4.8, 0.6),
            "custo": (2500, 200),
            "agua": (600, 60),
            "k": (100, 12),
            "p": (90, 10),
            "horas": (12, 1.8),
        },
        "Milho_Safrinha": {
            "prod": (5.5, 0.7),
            "custo": (2800, 250),
            "agua": (700, 70),
            "k": (120, 15),
            "p": (100, 12),
            "horas": (15, 2.0),
        },
    }

    def generate_data(params, size):
        return {
            "produtividade_ton_ha": np.random.normal(
                loc=params["prod"][0], scale=params["prod"][1], size=size
            ),
            "custo_ha": np.random.normal(
                loc=params["custo"][0], scale=params["custo"][1], size=size
            ),
            "uso_agua_m3_ha": np.random.normal(
                loc=params["agua"][0], scale=params["agua"][1], size=size
            ),
            "demanda_k_kg_ha": np.random.normal(
                loc=params["k"][0], scale=params["k"][1], size=size
            ),
            "demanda_p_kg_ha": np.random.normal(
                loc=params["p"][0], scale=params["p"][1], size=size
            ),
            "horas_maquina_ha": np.random.normal(
                loc=params["horas"][0], scale=params["horas"][1], size=size
            ),
        }

    for cultura, params in culturas_params.items():
        mask = df["cultura"] == cultura
        for col, values in generate_data(params, df[mask].shape[0]).items():
            df.loc[mask, col] = values

    # Debug: mostrar amostra e estatísticas resumidas
    print_and_log('Dados mock gerados com sucesso. Amostra das primeiras linhas:')
    sample = df.head(5).to_json(orient='records', force_ascii=False)
    print_and_log(sample)
    stats = df.describe().to_json(force_ascii=False)
    print_and_log('Estatísticas resumidas:')
    print_and_log(stats)
    return df


def setup_model(params, resources):
    # Forma mais robusta de extrair parâmetros, usando .loc para tudo
    l_sr = params.loc["Soja_Resistente", "lucro_ha"]
    l_sp = params.loc["Soja_Produtiva", "lucro_ha"]
    l_ms = params.loc["Milho_Safrinha", "lucro_ha"]

    c_sr = params.loc["Soja_Resistente", "custo_ha"]
    c_sp = params.loc["Soja_Produtiva", "custo_ha"]
    c_ms = params.loc["Milho_Safrinha", "custo_ha"]

    a_sr = params.loc["Soja_Resistente", "uso_agua_m3_ha"]
    a_sp = params.loc["Soja_Produtiva", "uso_agua_m3_ha"]
    a_ms = params.loc["Milho_Safrinha", "uso_agua_m3_ha"]

    k_sr, p_sr = params.loc["Soja_Resistente", ["demanda_k_kg_ha", "demanda_p_kg_ha"]]
    k_sp, p_sp = params.loc["Soja_Produtiva", ["demanda_k_kg_ha", "demanda_p_kg_ha"]]
    k_ms, p_ms = params.loc["Milho_Safrinha", ["demanda_k_kg_ha", "demanda_p_kg_ha"]]

    h_sr = params.loc["Soja_Resistente", "horas_maquina_ha"]
    h_sp = params.loc["Soja_Produtiva", "horas_maquina_ha"]
    h_ms = params.loc["Milho_Safrinha", "horas_maquina_ha"]

    prod_sr = params.loc["Soja_Resistente", "produtividade_ton_ha"]
    prod_sp = params.loc["Soja_Produtiva", "produtividade_ton_ha"]
    prod_ms = params.loc["Milho_Safrinha", "produtividade_ton_ha"]

    # O resto da função continua igual...
    modelo = LpProblem(name="Otimizacao_Mix_Culturas_Risco", sense=LpMaximize)

    x_sr = LpVariable("Hectares_Soja_Resistente", lowBound=0, cat="Continuous")
    x_sp = LpVariable("Hectares_Soja_Produtiva", lowBound=0, cat="Continuous")
    x_ms = LpVariable("Hectares_Milho_Safrinha", lowBound=0, cat="Continuous")

    # Gestão de Risco
    area_minima_ha = resources["AREA_NAO_COMPACTADA_HA"] * 0.15  # Usar config
    modelo += (x_sr >= area_minima_ha, "Minimo_Soja_Resistente")
    modelo += (x_sp >= area_minima_ha, "Minimo_Soja_Produtiva")
    modelo += (x_ms >= area_minima_ha, "Minimo_Milho_Safrinha")

    area_maxima_sp = resources["AREA_NAO_COMPACTADA_HA"] * 0.60
    modelo += (x_sp <= area_maxima_sp, "Risco_Maximo_Soja_Produtiva")

    # Função Objetivo
    modelo += lpSum([l_sr * x_sr, l_sp * x_sp, l_ms * x_ms]), "Lucro_Total"

    # Restrições
    modelo += (
        x_sr + x_sp + x_ms <= resources["AREA_TOTAL_DISPONIVEL_HA"],
        "Restricao_Area_Total",
    )
    modelo += (
        x_sr + x_sp + x_ms <= resources["AREA_NAO_COMPACTADA_HA"],
        "Restricao_Area_Nao_Compactada",
    )
    modelo += (
        c_sr * x_sr + c_sp * x_sp + c_ms * x_ms
        <= resources["ORCAMENTO_TOTAL_DISPONIVEL"],
        "Restricao_Orcamento",
    )
    modelo += (
        a_sr * x_sr + a_sp * x_sp + a_ms * x_ms
        <= resources["AGUA_TOTAL_DISPONIVEL_M3"],
        "Restricao_Agua",
    )
    modelo += (
        k_sr * x_sr + k_sp * x_sp + k_ms * x_ms <= resources["POTASSIO_DISPONIVEL_KG"],
        "Restricao_Potassio",
    )
    modelo += (
        p_sr * x_sr + p_sp * x_sp + p_ms * x_ms <= resources["FOSFORO_DISPONIVEL_KG"],
        "Restricao_Fosforo",
    )
    modelo += (
        h_sr * x_sr + h_sp * x_sp + h_ms * x_ms
        <= resources["HORAS_MAQUINA_DISPONIVEIS"],
        "Restricao_Horas_Maquina",
    )
    modelo += (
        prod_sr * x_sr + prod_sp * x_sp + prod_ms * x_ms
        <= resources["CAPACIDADE_SILO_TON"],
        "Restricao_Armazenagem",
    )

    # Debug: imprimir parâmetros usados no modelo
    print_and_log('Parâmetros do modelo (médias por cultura):')
    try:
        print_and_log(params.to_json(force_ascii=False))
    except Exception:
        print_and_log(str(params))
    print_and_log(f'Recursos carregados: {resources}')
    print_and_log("Modelo de otimização configurado.")
    return modelo, (x_sr, x_sp, x_ms)


def run_optimization(cfg: DictConfig):
    df = generate_mock_data(cfg.params.n_dados, cfg.params.seed)
    params = df.groupby("cultura").mean()
    params["lucro_ha"] = 0.0
    params.loc["Soja_Resistente", "lucro_ha"] = (
        params.loc["Soja_Resistente", "produtividade_ton_ha"] * cfg.params.preco_soja
    ) - params.loc["Soja_Resistente", "custo_ha"]
    params.loc["Soja_Produtiva", "lucro_ha"] = (
        params.loc["Soja_Produtiva", "produtividade_ton_ha"] * cfg.params.preco_soja
    ) - params.loc["Soja_Produtiva", "custo_ha"]
    params.loc["Milho_Safrinha", "lucro_ha"] = (
        params.loc["Milho_Safrinha", "produtividade_ton_ha"] * cfg.params.preco_milho
    ) - params.loc["Milho_Safrinha", "custo_ha"]

    modelo, variables = setup_model(params, cfg.resources)
    print_and_log('Iniciando solução do modelo...')
    solve_status = modelo.solve()
    status = LpStatus[modelo.status]
    print_and_log(f"Status do solver (raw): {solve_status}")
    print_and_log(f"Status da otimização: {status}")

    # Debug: valores das variáveis de decisão e preços sombra
    plantio = {v.name: v.varValue for v in variables} if status == "Optimal" else None
    shadow = {name: getattr(con, 'pi', None) for name, con in modelo.constraints.items()}
    print_and_log('Plantio (resultado):')
    print_and_log(str(plantio))
    print_and_log('Preços sombra:')
    print_and_log(str(shadow))

    results = {
        "status": status,
        "lucro": value(modelo.objective) if status == "Optimal" else None,
        "plantio": plantio,
        "shadow_prices": shadow,
    }
    return results


In [4]:
# 4. Executando e Coletando Todos os Cenários (carregamento e merge seguro de YAML)
import pandas as pd
import os
from hydra import initialize
from omegaconf import OmegaConf

# Lista de todos os cenários que queremos executar
scenario_names = [
    "base",
    "seca",
    "conservador",
    "agressivo",
    "preco_alto_soja",
    "crise_fertilizantes",
]
all_results = {}

# Carrega a base uma vez
base_cfg_path = os.path.join('config', 'cenario', 'base.yaml')
base_cfg = OmegaConf.load(base_cfg_path)

# Arquivo unificado solicitado para todas as saídas
MASTER_LOG = 'outputs-otimizacao_mix_culturas_v3_gestao_risco.txt'

for name in scenario_names:
    print_and_log(f"\n{'='*20} EXECUTANDO CENÁRIO: {name.upper()} {'='*20}", logfile=MASTER_LOG)
    if name == 'base':
        cfg = base_cfg
    else:
        override_path = os.path.join('config', 'cenario', f"{name}.yaml")
        if os.path.exists(override_path):
            override_cfg = OmegaConf.load(override_path)
            # Merge: valores do override substituem os da base
            cfg = OmegaConf.merge(base_cfg, override_cfg)
            print_and_log(f"Config '{name}' carregada e mesclada com sucesso: {cfg}", logfile=MASTER_LOG)
        else:
            print_and_log(f"Arquivo de cenário não encontrado: {override_path}. Usando base.", logfile=MASTER_LOG)
            cfg = base_cfg

    # Para compatibilidade com o código existente que espera DictConfig tipo hydra/omegaconf
    res = run_optimization(cfg)
    all_results[name] = res
    # também gravar resumo por cenário no arquivo mestre
    try:
        with open(MASTER_LOG, 'a', encoding='utf-8') as mf:
            mf.write('\n' + '-'*60 + '\n')
            mf.write(f"Cenário: {name}\n")
            mf.write(f"Status: {res.get('status')}\n")
            mf.write(f"Lucro: {res.get('lucro')}\n")
            mf.write(f"Plantio: {res.get('plantio')}\n")
            mf.write(f"Shadow Prices: {res.get('shadow_prices')}\n")
    except Exception as e:
        print_and_log(f'Erro ao gravar no arquivo mestre: {e}', logfile=MASTER_LOG)

print_and_log("\nTODOS OS CENÁRIOS FORAM EXECUTADOS COM SUCESSO.", logfile=MASTER_LOG)

# 5. Análise Consolidada com Pandas (Muito mais poderosa!)
processed_data = []
for scenario, result in all_results.items():
    if result['status'] == 'Optimal':
        plantio = result.get('plantio', {})
        processed_data.append({
            'Cenário': scenario,
            'Lucro Total': result.get('lucro'),
            'Soja Resistente (ha)': plantio.get('Hectares_Soja_Resistente'),
            'Soja Produtiva (ha)': plantio.get('Hectares_Soja_Produtiva'),
            'Milho Safrinha (ha)': plantio.get('Hectares_Milho_Safrinha'),
            'Shadow Price (Área)': result.get('shadow_prices', {}).get('Restricao_Area_Nao_Compactada'),
            'Shadow Price (Água)': result.get('shadow_prices', {}).get('Restricao_Agua'),
            'Status': result.get('status')
        })
    else:
        processed_data.append({
            'Cenário': scenario,
            'Status': result.get('status'),
            'Lucro Total': None, 'Soja Resistente (ha)': None, 'Soja Produtiva (ha)': None,
            'Milho Safrinha (ha)': None, 'Shadow Price (Área)': None, 'Shadow Price (Água)': None
        })

# Cria um DataFrame do Pandas para fácil visualização e análise
df_results = pd.DataFrame(processed_data)

print_and_log("\n--- TABELA RESUMO DOS CENÁRIOS ---", logfile=MASTER_LOG)
print_and_log(df_results.to_string(), logfile=MASTER_LOG)

# Exportar resumo CSV para fácil análise posterior (além do master log)
try:
    df_results.to_csv('resumo_cenarios.csv', index=False, sep=';', decimal=',')
    print_and_log('Resumo dos cenários exportado para resumo_cenarios.csv', logfile=MASTER_LOG)
except Exception as e:
    print_and_log(f'Erro ao exportar resumo_cenarios.csv: {e}', logfile=MASTER_LOG)

# 6. Dashboard Interativo Aprimorado com Todos os Cenários
import dash
from dash import html, dcc, Input, Output
import plotly.express as px
import plotly.graph_objects as go

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Dashboard de Análise de Cenários de Otimização'),
    
    html.Label('Selecione um Cenário:'),
    dcc.Dropdown(
        id='scenario-dropdown',
        options=[{'label': name.capitalize(), 'value': name} for name in all_results.keys()],
        value='base'
    ),
    
    dcc.Graph(id='plantio-graph'),
    
    html.H3('Preços Sombra (Valor de 1 unidade extra de recurso)'),
    html.Table(id='shadow-table')
])

@app.callback(
    [Output('plantio-graph', 'figure'), Output('shadow-table', 'children')],
    [Input('scenario-dropdown', 'value')]
)
def update_dashboard(scenario_name):
    res = all_results.get(scenario_name)
    if not res or res['status'] != 'Optimal':
        fig = go.Figure().update_layout(title=f'Cenário "{scenario_name.capitalize()}" não possui solução ótima.')
        return fig, []

    plantio_data = res.get('plantio', {})
    culturas = ['Soja Resistente', 'Soja Produtiva', 'Milho Safrinha']
    hectares = [
        plantio_data.get('Hectares_Soja_Resistente', 0),
        plantio_data.get('Hectares_Soja_Produtiva', 0),
        plantio_data.get('Hectares_Milho_Safrinha', 0)
    ]

    fig = px.bar(x=culturas, y=hectares, title=f"Plano de Plantio - Cenário {scenario_name.capitalize()} (Lucro: R$ {res.get('lucro',0):,.2f})", labels={'x':'Cultura','y':'Hectares (ha)'})

    table_header = [html.Thead(html.Tr([html.Th('Recurso'), html.Th('Preço Sombra (R$)')]))]
    rows = []
    for name, price in res.get('shadow_prices', {}).items():
        if price is not None and abs(price) > 0.001:
             rows.append(html.Tr([html.Td(name), html.Td(f'{price:,.2f}')]))
    table_body = [html.Tbody(rows)]
    return fig, table_header + table_body

print_and_log('Dashboard definido. Execute app.run_server() em um script separado para visualizar.', logfile=MASTER_LOG)


[2025-10-16 15:42:43] 
==================== EXECUTANDO CENÁRIO: BASE ====================
[2025-10-16 15:42:43] Dados mock gerados com sucesso. Amostra das primeiras linhas:
[2025-10-16 15:42:43] [{"id_talhao":1,"cultura":"Milho_Safrinha","produtividade_ton_ha":5.2898146975,"custo_ha":3166.429310426,"uso_agua_m3_ha":757.3338456894,"demanda_k_kg_ha":128.2676505354,"demanda_p_kg_ha":102.3053712049,"horas_maquina_ha":14.3371977926},{"id_talhao":2,"cultura":"Soja_Resistente","produtividade_ton_ha":2.8403260554,"custo_ha":1734.2119190614,"uso_agua_m3_ha":430.0444073899,"demanda_k_kg_ha":70.0360831083,"demanda_p_kg_ha":64.7504323609,"horas_maquina_ha":9.693557},{"id_talhao":3,"cultura":"Milho_Safrinha","produtividade_ton_ha":4.5891231629,"custo_ha":2601.484435741,"uso_agua_m3_ha":708.0206431971,"demanda_k_kg_ha":118.3480263633,"demanda_p_kg_ha":101.1328834408,"horas_maquina_ha":15.8967940331},{"id_talhao":4,"cultura":"Milho_Safrinha","produtividade_ton_ha":4.6262106431,"custo_ha":2487.099101

In [5]:
# 5. Visualize Results with Plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_comparison(results_base, results_seca):
    if not results_base or not results_seca:
        print_and_log('Dados para comparação incompletos; ignorando plot de comparação.')
        return

    if results_base.get('status') == 'Optimal' and results_seca.get('status') == 'Optimal':
        fig = make_subplots(rows=1, cols=2, subplot_titles=('Plano de Plantio - Base', 'Plano de Plantio - Seca'))
        
        culturas = ['Soja Resistente', 'Soja Produtiva', 'Milho Safrinha']
        base_values = [
            results_base['plantio'].get('Hectares_Soja_Resistente', 0),
            results_base['plantio'].get('Hectares_Soja_Produtiva', 0),
            results_base['plantio'].get('Hectares_Milho_Safrinha', 0)
        ]
        seca_values = [
            results_seca['plantio'].get('Hectares_Soja_Resistente', 0),
            results_seca['plantio'].get('Hectares_Soja_Produtiva', 0),
            results_seca['plantio'].get('Hectares_Milho_Safrinha', 0)
        ]
        
        fig.add_trace(go.Bar(x=culturas, y=base_values, name='Base'), row=1, col=1)
        fig.add_trace(go.Bar(x=culturas, y=seca_values, name='Seca'), row=1, col=2)
        
        fig.update_layout(title_text='Comparação de Planos de Plantio')
        fig.show()
    else:
        print_and_log('Resultados não ótimos para plotar.')

# Substitui a chamada direta anterior por uma versão segura que usa all_results
results_base = all_results.get('base') if 'all_results' in globals() else None
results_seca = all_results.get('seca') if 'all_results' in globals() else None
plot_comparison(results_base, results_seca)


In [6]:
# 6. Create Dash Dashboard
import dash
from dash import html, dcc, Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash


app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Dashboard de Otimização de Culturas'),
    dcc.Dropdown(
        id='scenario-dropdown',
        options=[
            {'label': 'Base', 'value': 'base'},
            {'label': 'Seca', 'value': 'seca'}
        ],
        value='base'
    ),
    dcc.Graph(id='plantio-graph'),
    html.Table(id='shadow-table')
])

@app.callback(
    [Output('plantio-graph', 'figure'), Output('shadow-table', 'children')],
    [Input('scenario-dropdown', 'value')]
)
def update_dashboard(scenario):
    if scenario == 'base':
        res = results_base
    else:
        res = results_seca
    
    if res['status'] == 'Optimal':
        fig = px.bar(x=['Soja Resistente', 'Soja Produtiva', 'Milho Safrinha'], 
                     y=[res['plantio']['Hectares_Soja_Resistente'], res['plantio']['Hectares_Soja_Produtiva'], res['plantio']['Hectares_Milho_Safrinha']],
                     title=f'Plano de Plantio - {scenario.capitalize()}')
        
        table = [html.Tr([html.Th('Recurso'), html.Th('Preço Sombra')])]
        for name, price in res['shadow_prices'].items():
            table.append(html.Tr([html.Td(name), html.Td(f'R$ {price:.2f}' if price else 'N/A')]))
        
        return fig, table
    else:
        return {}, []

# Para notebook, não executar app.run_server(), apenas definir
print_and_log('Dashboard definido. Execute app.run_server() em um script separado para visualizar.')

app.run(mode="inline", debug=True)

[2025-10-16 15:42:43] Dashboard definido. Execute app.run_server() em um script separado para visualizar.


In [7]:
# 7. Export Results to Spreadsheets
import pandas as pd


def export_results(results_base, results_seca, master_log='outputs-otimizacao_mix_culturas_v3_gestao_risco.txt'):
    # Exportar plantio
    plantio_df = pd.DataFrame(
        {
            "Cenário": ["Base", "Seca"],
            "Soja Resistente": [
                results_base["plantio"]["Hectares_Soja_Resistente"],
                results_seca["plantio"]["Hectares_Soja_Resistente"],
            ],
            "Soja Produtiva": [
                results_base["plantio"]["Hectares_Soja_Produtiva"],
                results_seca["plantio"]["Hectares_Soja_Produtiva"],
            ],
            "Milho Safrinha": [
                results_base["plantio"]["Hectares_Milho_Safrinha"],
                results_seca["plantio"]["Hectares_Milho_Safrinha"],
            ],
            "Lucro": [results_base["lucro"], results_seca["lucro"]],
        }
    )
    # ALTERADO: to_excel('resultados_plantio.xlsx', index=False) para to_csv()
    plantio_df.to_csv("resultados_plantio.csv", index=False, decimal=",", sep=";")

    # Exportar shadow prices
    shadow_df = pd.DataFrame(
        {
            "Cenário": ["Base"] * len(results_base["shadow_prices"])
            + ["Seca"] * len(results_seca["shadow_prices"]),
            "Recurso": list(results_base["shadow_prices"].keys())
            + list(results_seca["shadow_prices"].keys()),
            "Preço Sombra": list(results_base["shadow_prices"].values())
            + list(results_seca["shadow_prices"].values()),
        }
    )
    # ALTERADO: to_excel('resultados_shadow.xlsx', index=False) para to_csv()
    shadow_df.to_csv("resultados_shadow.csv", index=False, decimal=",", sep=";")

    # Também anotar no arquivo mestre tudo o que foi exportado
    try:
        with open(master_log, 'a', encoding='utf-8') as mf:
            mf.write('\n' + '='*40 + '\n')
            mf.write('Exportação de resultados (plantio e preços sombra)\n')
            mf.write('\nPlantio CSV (resultados_plantio.csv):\n')
            mf.write(plantio_df.to_string(index=False) + '\n')
            mf.write('\nShadow Prices CSV (resultados_shadow.csv):\n')
            mf.write(shadow_df.to_string(index=False) + '\n')
    except Exception as e:
        print_and_log(f'Erro ao gravar exportação no arquivo mestre: {e}', logfile=master_log)

    print_and_log("Resultados exportados para CSV.", logfile=master_log)


# Assumindo que results_base, results_seca e print_and_log estão definidos
# export_results(results_base, results_seca)


In [8]:
# 8. Simular cenários para propriedades pequenas, médias e grandes com base em Módulos Fiscais (MF)
# Vamos considerar MF = 5 ha e MF = 110 ha e criar intervalos para pequena/média/grande
from copy import deepcopy

# Definir área base usada originalmente (para escalonamento proporcional)
AREA_BASE_HA = 500

def generate_property_configs(mf_hectares):
    # Retorna 3 configurações (pequena, media, grande) baseadas em MF
    configs = {}
    # Pequena: 1 a 4 MF -> usar 1 MF como mínimo e 4 como máximo
    peque_min = 1 * mf_hectares
    peque_max = 4 * mf_hectares
    # Média: >4 até 15 MF
    media_min = (4 + 1) * mf_hectares
    media_max = 15 * mf_hectares
    # Grande: >15 MF -> usar 16 como mínimo
    grande_min = 16 * mf_hectares
    # Para máximo vamos definir como grande_min * 10 (valor arbitrário para simulação)
    grande_max = grande_min * 10

    def scale_resources(area_ha):
        # escala todos os recursos proporcionalmente à área em relação a AREA_BASE_HA
        factor = area_ha / AREA_BASE_HA
        return {
            'AREA_TOTAL_DISPONIVEL_HA': int(area_ha),
            'ORCAMENTO_TOTAL_DISPONIVEL': int(1100000 * factor),
            'AGUA_TOTAL_DISPONIVEL_M3': int(250000 * factor),
            'POTASSIO_DISPONIVEL_KG': int(45000 * factor),
            'FOSFORO_DISPONIVEL_KG': int(42000 * factor),
            'AREA_NAO_COMPACTADA_HA': int(area_ha),
            'HORAS_MAQUINA_DISPONIVEIS': int(6000 * factor),
            'CAPACIDADE_SILO_TON': int(2500 * factor),
        }

    # Pequena: usar média do intervalo pequena
    peque_area = int((peque_min + peque_max) / 2)
    configs['pequena'] = scale_resources(peque_area)

    # Média: média do intervalo média
    media_area = int((media_min + media_max) / 2)
    configs['media'] = scale_resources(media_area)

    # Grande: média entre grande_min e grande_max
    grande_area = int((grande_min + grande_max) / 2)
    configs['grande'] = scale_resources(grande_area)

    return configs


def run_property_simulations(mf_values=(5,110), output_file='resultados_propriedades_simulacao.txt'):
    all_results = {}
    # abrir arquivo único para escrita (append para não sobrescrever se re-executar)
    with open(output_file, 'a', encoding='utf-8') as fout:
        from datetime import datetime
        fout.write('\n' + '='*80 + '\n')
        fout.write(f"Simulações iniciadas em {datetime.now().isoformat(sep=' ', timespec='seconds')}\n")
        fout.write('='*80 + '\n')

        for mf in mf_values:
            fout.write(f"\n--- MF = {mf} ha ---\n")
            print_and_log(f'---- Simulações para MF = {mf} ha ----')
            configs = generate_property_configs(mf)
            all_results[mf] = {}
            for tamanho, resources in configs.items():
                fout.write(f"\n-- {tamanho.upper()} --\n")
                fout.write(f"Recursos: {resources}\n")
                # Construir um DictConfig-like mínimo esperado por run_optimization
                cfg = DictConfig({'params': {'n_dados': 1500, 'seed': 42, 'preco_soja': 2200, 'preco_milho': 1300}, 'resources': resources})
                print_and_log(f'Executando cenário {tamanho} (MF={mf}): recursos={resources}')
                try:
                    res = run_optimization(cfg)
                except Exception as e:
                    print_and_log(f'Erro ao executar otimização para MF={mf} tamanho={tamanho}: {e}')
                    res = {'status': 'Error', 'error': str(e)}
                all_results[mf][tamanho] = res
                # Escrever resultado no arquivo único
                fout.write("Resultado:\n")
                fout.write(str(res) + '\n')
                fout.write('-'*40 + '\n')
                print_and_log(f'Resultados para MF={mf} tamanho={tamanho} adicionados em {output_file}')

        fout.write('\nSimulações finalizadas.\n')
        fout.write('='*80 + '\n')
    return all_results

# Executar simulações e gravar em um único arquivo
sim_results = run_property_simulations((5,110))
print_and_log('Simulações de propriedades concluídas. Resumo:')
print_and_log(str(sim_results))

# Exportar um resumo consolidado (JSON) também
import json
with open('resumo_simulacoes_propriedades.json', 'w', encoding='utf-8') as f:
    json.dump(sim_results, f, indent=2, ensure_ascii=False)
print_and_log('Resumo consolidado exportado para resumo_simulacoes_propriedades.json')


[2025-10-16 15:42:43] ---- Simulações para MF = 5 ha ----
[2025-10-16 15:42:43] Executando cenário pequena (MF=5): recursos={'AREA_TOTAL_DISPONIVEL_HA': 12, 'ORCAMENTO_TOTAL_DISPONIVEL': 26400, 'AGUA_TOTAL_DISPONIVEL_M3': 6000, 'POTASSIO_DISPONIVEL_KG': 1080, 'FOSFORO_DISPONIVEL_KG': 1008, 'AREA_NAO_COMPACTADA_HA': 12, 'HORAS_MAQUINA_DISPONIVEIS': 144, 'CAPACIDADE_SILO_TON': 60}
[2025-10-16 15:42:43] Dados mock gerados com sucesso. Amostra das primeiras linhas:
[2025-10-16 15:42:43] [{"id_talhao":1,"cultura":"Milho_Safrinha","produtividade_ton_ha":5.2898146975,"custo_ha":3166.429310426,"uso_agua_m3_ha":757.3338456894,"demanda_k_kg_ha":128.2676505354,"demanda_p_kg_ha":102.3053712049,"horas_maquina_ha":14.3371977926},{"id_talhao":2,"cultura":"Soja_Resistente","produtividade_ton_ha":2.8403260554,"custo_ha":1734.2119190614,"uso_agua_m3_ha":430.0444073899,"demanda_k_kg_ha":70.0360831083,"demanda_p_kg_ha":64.7504323609,"horas_maquina_ha":9.693557},{"id_talhao":3,"cultura":"Milho_Safrinha","p